In [1]:
import numpy as np
import pandas as pd

import lightgbm as lgb
import xgboost as xgb
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [61]:
train = pd.read_csv('adult.data.txt')
test = pd.read_csv('adult.test.txt')

In [5]:
train.head()

,age,workclass,fnlwgt,education,education-num,marital_Status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,Income
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [4]:
train.columns = ['age','workclass','fnlwgt','education','education-num','marital_Status','occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country','Income']

In [7]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [18]:
## Label Encoding Income variable as it is >50000 & <=50000

In [8]:
l = LabelEncoder()

In [9]:
l.fit(train.Income)

LabelEncoder()

In [10]:
l.classes_

array([' <=50K', ' >50K'], dtype=object)

In [14]:
train.Income = pd.Series(l.transform(train.Income))

In [16]:
train.describe()

,age,fnlwgt,education-num,capital_gain,capital_loss,hours_per_week,Income
count,32560.000000,3.256000e+04,32560.000000,32560.000000,32560.000000,32560.000000,32560.000000
mean,38.581634,1.897818e+05,10.080590,1077.615172,87.306511,40.437469,0.240817
std,13.640642,1.055498e+05,2.572709,7385.402999,402.966116,12.347618,0.427586
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000,0.000000
25%,28.000000,1.178315e+05,9.000000,0.000000,0.000000,40.000000,0.000000
50%,37.000000,1.783630e+05,10.000000,0.000000,0.000000,40.000000,0.000000
75%,48.000000,2.370545e+05,12.000000,0.000000,0.000000,45.000000,0.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000,1.000000


In [17]:
train.Income.value_counts()

0    24719
1     7841
Name: Income, dtype: int64

In [20]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32560 entries, 0 to 32559
Data columns (total 15 columns):
age               32560 non-null int64
workclass         32560 non-null object
fnlwgt            32560 non-null int64
education         32560 non-null object
education-num     32560 non-null int64
marital_Status    32560 non-null object
occupation        32560 non-null object
relationship      32560 non-null object
race              32560 non-null object
sex               32560 non-null object
capital_gain      32560 non-null int64
capital_loss      32560 non-null int64
hours_per_week    32560 non-null int64
native_country    32560 non-null object
Income            32560 non-null int64
dtypes: int64(7), object(8)
memory usage: 3.7+ MB


In [21]:
bn = train.select_dtypes(include=['object'])

In [23]:
bn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32560 entries, 0 to 32559
Data columns (total 8 columns):
workclass         32560 non-null object
education         32560 non-null object
marital_Status    32560 non-null object
occupation        32560 non-null object
relationship      32560 non-null object
race              32560 non-null object
sex               32560 non-null object
native_country    32560 non-null object
dtypes: object(8)
memory usage: 2.0+ MB


In [24]:
## One hot encoding of Categorical variables

In [25]:
bn = pd.get_dummies(train, drop_first= True)

In [29]:
bn.head()

,age,fnlwgt,education-num,capital_gain,capital_loss,hours_per_week,Income,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,...,native_country_ Portugal,native_country_ Puerto-Rico,native_country_ Scotland,native_country_ South,native_country_ Taiwan,native_country_ Thailand,native_country_ Trinadad&Tobago,native_country_ United-States,native_country_ Vietnam,native_country_ Yugoslavia
0,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,215646,9,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,53,234721,7,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,28,338409,13,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,37,284582,14,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [30]:
train.native_country.nunique()

42

In [31]:
train.sex.nunique()

2

In [33]:
bn['sex_ Male'].value_counts()

1    21789
0    10771
Name: sex_ Male, dtype: int64

In [34]:
train.sex.value_counts()

 Male      21789
 Female    10771
Name: sex, dtype: int64

In [42]:
for i in train.select_dtypes(include=[object]):
    print(i)

workclass
education
marital_Status
occupation
relationship
race
sex
native_country


In [44]:
pd.get_dummies(train,columns = train.select_dtypes(include = [object]),drop_first=True)

,age,fnlwgt,education-num,capital_gain,capital_loss,hours_per_week,Income,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,...,native_country_ Portugal,native_country_ Puerto-Rico,native_country_ Scotland,native_country_ South,native_country_ Taiwan,native_country_ Thailand,native_country_ Trinadad&Tobago,native_country_ United-States,native_country_ Vietnam,native_country_ Yugoslavia
0,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,215646,9,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,53,234721,7,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,28,338409,13,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,37,284582,14,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,49,160187,5,0,0,16,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,52,209642,9,0,0,45,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7,31,45781,14,14084,0,50,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8,42,159449,13,5178,0,40,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,37,280464,10,0,0,80,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [51]:
train1 = pd.get_dummies(train,columns = train.select_dtypes(include = [object]))

In [52]:
train1.shape

(32560, 109)

In [55]:
train1.columns.nunique()

109

In [56]:
## Crearting data in to train & Target variable

x = train1.drop('Income', axis = 1)
y = train1['Income']

In [57]:
x.shape

(32560, 108)

In [58]:
y.shape

(32560,)

In [69]:
## Creating test and  train splits
from sklearn.model_selection import train_test_split
x_train, x_test, y_train,y_test = train_test_split(x,y, test_size = .25)

## Applying XGBoost

In [70]:
dtrain = xgb.DMatrix(x_train, label = y_train)
dtest = xgb.DMatrix(x_test)

In [76]:
## Setting parameters for XGBoost

parameters = {'max_depth':7, 
              'learning_rate': 0.01, 
              'n_estimators':1000, 
              'objective' :'binary:logistic',
              'eval_metric' : 'auc',
              'eta' : .1}

In [78]:
## Training model

num_rounds = 50
from datetime import datetime
start = datetime.now()
xgb = xgb.train(parameters,dtrain, num_boost_round=num_rounds)
stop = datetime.now()

In [79]:
## Execution time of model
execution_time  = stop - start
execution_time

datetime.timedelta(0, 2, 360808)

In [81]:
## Predicting
ypred = xgb.predict(dtest)

In [82]:
ypred

array([0.31648478, 0.31000233, 0.35472158, ..., 0.69137245, 0.30638978,
       0.31000233], dtype=float32)

In [92]:
for i in range(0,8139): ## 8139 lenght of the y_test
    if ypred[i]>=.5:
        ypred[i]=1
    else:
        ypred[i]=0

In [94]:
from sklearn.metrics import accuracy_score

In [97]:
accuracy_xgb =  accuracy_score(y_test, ypred)
accuracy_xgb

0.8597051597051597

# Light GBM

In [98]:
lgb_train = lgb.Dataset(x_train, label=y_train)

In [100]:
## Parameters Setting

param = {'num_leaves':150, 
              'max_depth': 7, 
              'learning_rate':0.01,
              'n_estimators':1000,
              'max_bin':200}
param['metric'] = ['auc', 'binary_logloss']

In [101]:
## Training the model

num_rounds = 50
start  = datetime.now()
lgbm = lgb.train(param, lgb_train, num_boost_round= num_rounds)
stop = datetime.now()

In [102]:
## Executon time
stop - start

datetime.timedelta(0, 2, 593428)

In [103]:
ypred2 = lgbm.predict(x_test)
ypred2

array([ 0.09412101,  0.04969361,  0.21916802, ...,  0.9521127 ,
       -0.00247642,  0.10991933])

In [125]:
for i in range(0, len(ypred2)):
    if ypred2[i]>=.5:
        ypred2[i]=1
    else:
        ypred2[i]=0

In [126]:
len(y_test)

8140

In [127]:
set(ypred2)

{0.0, 1.0}

In [128]:
## Calculating Accuracy

lgbm_accuracy = accuracy_score( y_test,ypred2)
lgbm_accuracy

0.8723587223587224